# Histogram & Stats:

https://opensourceoptions.com/blog/zonal-statistics-algorithm-with-python-in-4-steps/
<br> https://ichi.pro/pt/algoritmo-de-estatistica-zonal-com-python-em-4-etapas-61754036348042

In [1]:
#___Gerar estatísticas dos requisições das rotas(excel)_____

import gdal # Gerenciar o raster
import ogr # Gerenciar o vetor
import os # Gerenciar arquivo
import numpy as np # Dados tabelados
import csv # Salvar arquivos em csv
import pandas as pd
import datetime
from tqdm import tqdm


## 1 - Definição de functions

In [2]:
def boundingBoxToOffsets(bbox, geot):
    col1 = int((bbox[0] - geot[0]) / geot[1])
    col2 = int((bbox[1] - geot[0]) / geot[1]) + 1
    row1 = int((bbox[3] - geot[3]) / geot[5])
    row2 = int((bbox[2] - geot[3]) / geot[5]) + 1
    return [row1, row2, col1, col2]


def geotFromOffsets(row_offset, col_offset, geot):
    new_geot = [
    geot[0] + (col_offset * geot[1]),
    geot[1],
    0.0,
    geot[3] + (row_offset * geot[5]),
    0.0,
    geot[5]
    ]
    return new_geot

#Estatísticas que serão avaliadas (id, mínimo, máximo, médio, desvio padrão, etc)
def setFeatureStats(fid, min, max, mean, median, sd, sum, count, names=["min", "max", "mean", "median", "sd", "sum", "count", "id"]):
    featstats = {
    names[0]: min,
    names[1]: max,
    names[2]: mean,
    names[3]: median,
    names[4]: sd,
    names[5]: sum,
    names[6]: count,
    names[7]: fid,
    }
    return featstats

In [3]:
# Recebe os dados de entrada como, arco e respectiva rota; encontra os arquivos vetoriais do arco, horário e respectiva 
# imagem GeoTiff; aplica estatística zonal

def zona_estatistica(rota,arco):
    # Arquivo vetorial do arco 
    fn_vetor = (r"arcos_desagregados\\"+rota+"\\"+arco+"\\"+str(arco)+".shp")
  
    # Hora Inicial da viagem
    fhand = pd.read_excel(r"...\Rotas_TomTom\rota_blocos\\"+rota+"\\"+rota+".xlsx",header = 10).set_index('Unnamed: 0')
    thand = (fhand["Unnamed: 1"]["Via Radares SP"])
    tempo_inicio= datetime.datetime.strptime(thand[4:9],"%H:%M")
    
    # Lista com hora acumulada para cada trecho
    fhand_2 = pd.read_excel(r"...\Rotas_TomTom\rota_blocos\\"+rota+"\\"+rota+".xlsx", sheet_name = "Route1-CumulativeTravelTimes")
    thand_2 = fhand_2["Unnamed: 3"][4:]
    
    # Acréscimo baseado no tempo cumulativo, garante que inicia em zero
    if index == 0: acr = datetime.timedelta(seconds = 0)
    else: acr = datetime.timedelta(seconds = thand_2.iloc[index])
    
    # Obtenção do raster (GeoTiff)
    fn_raster = (r"...\Precipitação\S-TORM 2019-02-16\bloco_tiff_2\\"+datetime.datetime.strftime(tempo_inicio+acr,"%H-%M")+".tif")
    #print(fn_raster)
    
    mem_driver = ogr.GetDriverByName("Memory")
    mem_driver_gdal = gdal.GetDriverByName("MEM")
    shp_name = "temp"

    # Carregar dados do raster e vetor
    raster_ds = gdal.Open(fn_raster)
    vetor_ds = ogr.Open(fn_vetor)
        
    # Carregar layer do vetor
    lyr = vetor_ds.GetLayer()
        
    # get geotransform and no data for raster
    geot = raster_ds.GetGeoTransform()
    nodata = raster_ds.GetRasterBand(1).GetNoDataValue()
    
    zstats = [] # empty list to hold the zonal statistics once they are calculated

    p_feat = lyr.GetNextFeature()
    niter = 0
        
    while p_feat:
        if p_feat.GetGeometryRef() is not None:
            if os.path.exists(shp_name):
                mem_driver.DeleteDataSource(shp_name)
            ttvetor_ds = mem_driver.CreateDataSource(shp_name)
            tp_lyr = ttvetor_ds.CreateLayer('polygons', None, ogr.wkbPolygon)
            tp_lyr.CreateFeature(p_feat.Clone())
            offsets = boundingBoxToOffsets(p_feat.GetGeometryRef().GetEnvelope(),\
            geot)
            new_geot = geotFromOffsets(offsets[0], offsets[2], geot)
                
            traster_ds = mem_driver_gdal.Create(\
            "", \
            offsets[3] - offsets[2], \
            offsets[1] - offsets[0], \
            1, \
            gdal.GDT_Byte)
                
            traster_ds.SetGeoTransform(new_geot)
            gdal.RasterizeLayer(traster_ds, [1], tp_lyr, burn_values=[1])
            tr_array = traster_ds.ReadAsArray()
                
            r_array = raster_ds.GetRasterBand(1).ReadAsArray(\
            offsets[2],\
            offsets[0],\
            offsets[3] - offsets[2],\
            offsets[1] - offsets[0])
        
            id = p_feat.GetFID()
        
            if r_array is not None:
                maskarray=np.ma.MaskedArray(\
                r_array,\
                mask=np.logical_or(r_array==nodata, np.logical_not(tr_array)))
            
                if maskarray is not None:
                    zstats.append(setFeatureStats(\
                    id,\
                    maskarray.min(),\
                    maskarray.max(),\
                    maskarray.mean(),\
                    np.ma.median(maskarray),\
                    maskarray.std(),\
                    maskarray.sum(),\
                    maskarray.count()))
                else:
                    zstats.append(setFeatureStats(\
                    id,\
                    nodata,\
                    nodata,\
                    nodata,\
                    nodata,\
                    nodata,\
                    nodata,\
                    nodata))
            else:
                zstats.append(setFeatureStats(\
                id,\
                nodata,\
                nodata,\
                nodata,\
                nodata,\
                nodata,\
                nodata,\
                nodata))
        
            ttvetor_ds = None
            tp_lyr = None
            traster_ds = None
        
            p_feat = lyr.GetNextFeature()
        
    return zstats

## 2 - Aplicação da estatística zonal

In [4]:
# Arquivos vetoriais (.shp): Dicionário com todas as rotas, contendo seus respectivos arcos
rotas = {}
for i in os.listdir(r"arcos_desagregados"):
    arcos = os.listdir(r"arcos_desagregados\\"+i)
    if "resultados" in arcos: arcos.remove("resultados")
    arcos.sort(key = lambda x: int(x.split("_")[1]))
    rotas[i]=arcos

In [6]:
for rota in tqdm(rotas):
    if (rota+".csv") in os.listdir("resultados_desagregados"): continue
    for arco in rotas[rota]:
        index = rotas[rota].index(arco) # Cria um índice para referência
        zstats = zona_estatistica(rota,arco) # Aplica Estatística Zonal
        
        # Agregando dados em um só csv
        if index == 0: df=pd.DataFrame(zstats)
        if index > 0: df = df.append(zstats[0], ignore_index=True, sort = False)
    
    # Agregando dados extras da planilha TomTom
    planilha_tomtom = pd.read_excel(r"...\Rotas_TomTom\rota_blocos\\"+rota+"\\"+rota+".xlsx", sheet_name = "Route1-CumulativeTravelTimes")
    
    df = df.rename(columns={"mean": "Precip. Media (dBZ)"})
    df["Precip. Media (mm/h)"] = ((df["Precip. Media (dBZ)"]/200)**(1/1.6)).fillna(0)
    df["Precip. Media Acum. (mm/h)"] = df["Precip. Media (mm/h)"].cumsum()
    
    df["Intensidade"] = pd.cut(df["Precip. Media (dBZ)"],[0,15,20,25,30,35,40,45,50,55,60,100],
           labels = ["Nao Chove","Garoa","Muito Fraca", "Fraca",
                     "Moderada Fraca","Moderada","Moderada Forte",
                     "Forte","Muito Forte","Muitissimo Forte","Extrema"])
    df["Intensidade Segregada"] = pd.cut(df["Precip. Media (dBZ)"],[0,15,30,45,60],
       labels = ["Nao Chove","Chuva Fraca","Chuva Moderada","Chuva Forte"])    
    
    df["Dist. Percorrida Acum. (m)"] = planilha_tomtom["Unnamed: 2"][4:].to_list()
    df["Dist. Percorrida Acum. (Km)"] = df["Dist. Percorrida Acum. (m)"]/1000
    df["Dist. Percorrida (Km)"] = df["Dist. Percorrida Acum. (Km)"]-df["Dist. Percorrida Acum. (Km)"].shift(1).fillna(0)
    df["Tempo de Viagem Acum. (s)"] = planilha_tomtom["Unnamed: 3"][4:].to_list()
    df["Tempo de Viagem (s)"] = df["Tempo de Viagem Acum. (s)"]-df["Tempo de Viagem Acum. (s)"].shift(1).fillna(0).to_list()
    
    # Cria nova coluna índice (rota)
    df["Arco"] = rotas[rota]
    df = (df.set_index("Arco")
          .drop(["sum","id","count","sd","median","Dist. Percorrida Acum. (m)","min","max"],axis = 1))
    
    # Salvar os resultados
    df.to_csv(r"resultados_desagregados\\"+rota+".csv")
    #break
df

100%|████████████████████████████████████████████████████████████████████████████| 1763/1763 [00:01<00:00, 1336.14it/s]


,Precip. Media (dBZ),Precip. Media (mm/h),Precip. Media Acum. (mm/h),Intensidade,Intensidade Segregada,Dist. Percorrida Acum. (Km),Dist. Percorrida (Km),Tempo de Viagem Acum. (s),Tempo de Viagem (s)
Arco,,,,,,,,,
arco_1,21.591982,0.248765,0.248765,Muito Fraca,Chuva Fraca,0.02113,0.02113,6.20,6.20
arco_2,21.591982,0.248765,0.497530,Muito Fraca,Chuva Fraca,0.06131,0.04018,24.98,18.78
arco_3,21.591982,0.248765,0.746295,Muito Fraca,Chuva Fraca,0.07806,0.01675,28.22,3.24
arco_4,21.591982,0.248765,0.995060,Muito Fraca,Chuva Fraca,0.08915,0.01109,29.97,1.75
arco_5,21.240948,0.246229,1.241289,Muito Fraca,Chuva Fraca,0.20247,0.11332,45.57,15.60
...,...,...,...,...,...,...,...,...,...
arco_360,31.909967,0.317549,77.968781,Moderada Fraca,Chuva Moderada,20.08744,0.07638,3009.20,5.49
arco_361,31.541531,0.315252,78.284033,Moderada Fraca,Chuva Moderada,20.13382,0.04638,3012.56,3.36
arco_362,31.541531,0.315252,78.599285,Moderada Fraca,Chuva Moderada,20.16629,0.03247,3014.96,2.40
